In [1]:
!git clone https://github.com/Hang-Ruan/CLIP_video.git

Cloning into 'CLIP_video'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 230 (delta 29), reused 0 (delta 0), pack-reused 187 (from 1)
Receiving objects: 100% (230/230), 1.51 MiB | 32.84 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [2]:
# From CLIP
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install opencv-python boto3 requests pandas

/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 6.8 MB/s eta 0:00:00


In [3]:
# Connet to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Comment if the file exits
# !wget -O /content/drive/MyDrive/Data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /content/drive/MyDrive/Data/msrvtt_data.zip -d /content/drive/MyDrive/Data/

In [4]:
DATA_PATH = "/content/drive/MyDrive/IDL/IDL Project/Data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/content/drive/MyDrive/IDL/IDL Project/Data/video"    # Directory of MSRVTT raw video

In [ ]:
# import pandas as pd

# # Load training dataset
# train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
# df = pd.read_csv(train_path)

# print("Size: ", len(df))
# print("Column names", df.columns.tolist())
# display(df.head())

# # Choose a subset
# def sample_subset(df, frac=0.1, random_state=42):
#     return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# # Set fraction to 0.01
# subset_df = sample_subset(df, frac=0.1)
# print("Size of subset：", len(subset_df))
# display(subset_df.head())

# # Save the subset
# subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
# subset_df.to_csv(subset_path, index=False)

# print("Subset CSV saved to:", subset_path)


In [5]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: NVIDIA L4


In [6]:
!wget -P ./modules https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt

--2025-04-28 22:25:40--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.59, 2620:1ec:bdf::59
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘./modules/ViT-B-32.pt’

ViT-B-32.pt         100%[===================>] 337.58M  59.6MB/s    in 6.0s    

2025-04-28 22:25:47 (56.6 MB/s) - ‘./modules/ViT-B-32.pt’ saved [353976522/353976522]



In [7]:
torch.cuda.empty_cache()

In [8]:
# Updata changes to files. Comment if the files are not changed
!cd CLIP_video && git pull

Already up to date.


In [ ]:
# !pip install ffmpeg
# !python /content/CLIP_video/preprocess/compress_video.py --input_root "/content/drive/MyDrive/IDL/IDL Project/Data/video" --output_root "/content/drive/MyDrive/IDL/IDL Project/Data/compressed_video"

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=9498c042b24cf5969fdb159dc7167087b97508c177a9014a9f73de86d77d9814
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg
Total video need to process: 0
Begin with 12-core logical processor.
Compress finished, wait for checking files...


In [11]:
%%bash
export DATA_PATH="/content/drive/MyDrive/IDL/IDL Project/Data/msrvtt_data"
export VIDEO_PATH="/content/drive/MyDrive/IDL/IDL Project/Data/video"

python /content/CLIP_video/main_task_retrieval.py \
  --do_train \
  --num_thread_reader 8 \
  --epochs 20 \
  --batch_size 64 \
  --train_csv  "$DATA_PATH"/MSRVTT_train.subset.csv \
  --val_csv    "$DATA_PATH"/MSRVTT_JSFUSION_test.csv \
  --data_path  "$DATA_PATH"/MSRVTT_data.json \
  --features_path "$VIDEO_PATH" \
  --output_dir /content/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
  --lr 1e-4 \
  --max_words 32 \
  --max_frames 8 \
  --batch_size_val 8 \
  --datatype msrvtt \
  --expand_msrvtt_sentences \
  --feature_framerate 1 \
  --coef_lr 1e-3 \
  --freeze_layer_num 6 \
  --slice_framepos 9 \
  --loose_type \
  --linear_patch 2d \
  --sim_header seqLSTM \
  --pretrained_clip_name ViT-B/32 \
  --fp16 \
  --fp16_opt_level O1


04/29/2025 02:23:24 - INFO -   Effective parameters:
04/29/2025 02:23:24 - INFO -     <<< batch_size: 64
04/29/2025 02:23:24 - INFO -     <<< batch_size_val: 8
04/29/2025 02:23:24 - INFO -     <<< cache_dir: 
04/29/2025 02:23:24 - INFO -     <<< coef_lr: 0.001
04/29/2025 02:23:24 - INFO -     <<< cross_model: cross-base
04/29/2025 02:23:24 - INFO -     <<< cross_num_hidden_layers: 4
04/29/2025 02:23:24 - INFO -     <<< data_path: /content/drive/MyDrive/IDL/IDL Project/Data/msrvtt_data/MSRVTT_data.json
04/29/2025 02:23:24 - INFO -     <<< datatype: msrvtt
04/29/2025 02:23:24 - INFO -     <<< do_eval: False
04/29/2025 02:23:24 - INFO -     <<< do_lower_case: False
04/29/2025 02:23:24 - INFO -     <<< do_pretrain: False
04/29/2025 02:23:24 - INFO -     <<< do_train: True
04/29/2025 02:23:24 - INFO -     <<< epochs: 20
04/29/2025 02:23:24 - INFO -     <<< eval_frame_order: 0
04/29/2025 02:23:24 - INFO -     <<< expand_msrvtt_sentences: True
04/29/2025 02:23:24 - INFO -     <<< feature_fram